In [1]:
import csv
import xml.etree.ElementTree as et
import numpy as np
import pandas as pd
from datetime import datetime as dt

In [2]:
tree = et.ElementTree(file = "f24_new.xml")
gameFile = tree.getroot()

In [3]:
#Print a string with the two teams, using %s and the attrib to dynamically fill the string
print ("{} vs {}".format(gameFile[0].attrib["home_team_name"], gameFile[0].attrib["away_team_name"]))

FC København vs Silkeborg IF


In [4]:
#Print a string with the two teams, using %s and the attrib to dynamically fill the string
print ("{} vs {} \nScore: {} - {}".format(gameFile[0].attrib["home_team_name"], gameFile[0].attrib["away_team_name"],
                                       gameFile[0].attrib["home_score"], gameFile[0].attrib["away_score"], ))

FC København vs Silkeborg IF 
Score: 4 - 2


In [5]:
#Print attributes for <EVENT>
gameFile[0][0].attrib

{'event_id': '1',
 'id': '2153299987',
 'last_modified': '2019-09-28T16:30:02',
 'min': '0',
 'outcome': '1',
 'period_id': '16',
 'sec': '0',
 'team_id': '418',
 'timestamp': '2019-09-28T15:12:42.953',
 'type_id': '34',
 'version': '1569684602204',
 'x': '0.0',
 'y': '0.0'}

In [6]:
#Print attributes for <EVENT QUALIFIERS>
gameFile[0][0][0].attrib

{'id': '2176906749',
 'qualifier_id': '227',
 'value': '0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0'}

In [7]:
#Making TEAM-ID into TEAM-NAME
team_dict = {gameFile[0].attrib["home_team_id"]: gameFile[0].attrib["home_team_name"],
            gameFile[0].attrib["away_team_id"]: gameFile[0].attrib["away_team_name"]}


print(team_dict)

{'569': 'FC København', '418': 'Silkeborg IF'}


In [8]:
#Create empty lists for the 8 columns we're collecting data for (PASSING)
x_origin = []
y_origin = []
x_destination = []
y_destination = []
outcome = []
minute = []
half = []
team = []

In [9]:
for game in gameFile:
    for event in game:
        if event.attrib.get("type_id") == '1':
            x_origin.append(event.attrib.get("x"))
            y_origin.append(event.attrib.get("y"))
            outcome.append(event.attrib.get("outcome"))
            minute.append(event.attrib.get("min"))
            half.append(event.attrib.get("period_id"))
            team.append(team_dict[event.attrib.get("team_id")])
            for qualifier in event:
                if qualifier.attrib.get("qualifier_id") == "140":
                    x_destination.append(qualifier.attrib.get("value"))
                if qualifier.attrib.get("qualifier_id") == "141":
                    y_destination.append(qualifier.attrib.get("value"))

In [10]:
print("The list is " + str(len(minute)) + " long and the 43rd entry is " + minute[42])

The list is 1150 long and the 43rd entry is 3


In [11]:
print(x_origin[6],y_origin[6],x_destination[6],y_destination[6],outcome[6])

69.9 100.0 59.6 89.0 1


In [12]:
#Create a list of our 8 columns/lists
column_titles = ["team", "half", "min", "x_origin", "y_origin", "x_destination", "y_destination", "outcome"]
            
#Use pd.DataFrame to create our table, assign the data in the order of our columns and give it the column titles above
final_table = pd.DataFrame(data=[team, half, minute, x_origin, y_origin, x_destination, y_destination, outcome], index=column_titles)

#Transpose, or flip, the table. Otherwise, our table will run from left to right, rather than top to bottom
final_table = final_table.T

#Show us the top 5 rows of the table
final_table.head()

,team,half,min,x_origin,y_origin,x_destination,y_destination,outcome
0,Silkeborg IF,1,0,50.0,49.9,47.3,50.5,1
1,Silkeborg IF,1,0,47.2,49.9,42.4,36.0,1
2,Silkeborg IF,1,0,42.0,28.8,45.0,4.3,1
3,Silkeborg IF,1,0,40.0,7.5,25.5,21.6,1
4,Silkeborg IF,1,0,22.7,25.1,7.8,47.3,1


In [13]:
#Make to floats

#Making the data match the field size of 105m x 68m
final_table['x_origin'] = (final_table['x_origin'].astype('float64'))/100*105
final_table['y_origin'] = (final_table['y_origin'].astype('float64'))/100*68
final_table['x_destination'] = (final_table['x_destination'].astype('float64'))/100*105
final_table['y_destination'] = (final_table['y_destination'].astype('float64'))/100*68

In [14]:
#Show us the top 5 rows of the table
final_table.head()

,team,half,min,x_origin,y_origin,x_destination,y_destination,outcome
0,Silkeborg IF,1,0,52.500,33.932,49.665,34.340,1
1,Silkeborg IF,1,0,49.560,33.932,44.520,24.480,1
2,Silkeborg IF,1,0,44.100,19.584,47.250,2.924,1
3,Silkeborg IF,1,0,42.000,5.100,26.775,14.688,1
4,Silkeborg IF,1,0,23.835,17.068,8.190,32.164,1


In [15]:
final_table.to_csv("pass_data.csv", index=False)